In [2]:
# Importando as libraries

import requests
import pandas as pd
import json
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, MetaData, Table
from sqlalchemy.orm import declarative_base, sessionmaker
import psycopg2


In [105]:
# Criando dicionários com os endpoints e seus descritivos separados por temas

indicadores_utilidade = {
    1207:  "PIB em R$ correntes",
    21774: "População",
    21775: "PIB per capita em R$ correntes",
    1619:  "Salário Mínimo",
    433	:  "Índice nacional de preços ao consumidor-amplo (IPCA)"
}

indicadores_emprego = {
    24369: "Taxa de desocupação - PNADC",
    24370: "Pessoas em idade de trabalhar - PNADC",
    24371: "Empregados no setor privado e público - PNADC",
    24372: "Empregados no setor público - PNADC",
    24373: "Empregado por conta própria - PNADC",
    24374: "Empregadores - PNADC",
    24375: "Empregados no setor privado - Com carteira - PNADC",
    24376: "Empregados no setor privado - Sem Carteira - PNADC",
    24377: "Empregados no setor privado - Total - PNADC",
    24378: "População na força de trabalho - PNADC",
    24379: "Pessoas ocupadas - PNADC",
    24380: "Pessoas desocupadas - PNADC",
    28543: "População ocupada com rendimento"
}

indicadores_rendimento_habitual = {
    24382: "Rendimento médio real habitual das pessoas ocupadas - Total - PNADC",
    24383: "Rendimento médio real habitual das pessoas ocupadas - Com carteira - PNADC",
    24384: "Rendimento médio real habitual das pessoas ocupadas - Sem carteira - PNADC",
    24385: "Rendimento médio real habitual das pessoas ocupadas - Setor privado - PNADC",
    24386: "Rendimento médio real habitual das pessoas ocupadas - Setor público - PNADC",
    24387: "Rendimento médio real habitual das pessoas ocupadas - Conta própria - PNADC",
    24388: "Rendimento médio real habitual das pessoas ocupadas - Empregadores - PNADC",
    24399: "Rendimento médio real habitual das pessoas ocupadas - Setor privado e público - PNADC"
}

In [106]:
# Definição das variáveis relevantes

# Dicionário com os grupos de endpoints e os temas respectivos
grupos_endpoints = {
    "utilidades": [1207, 21774, 21775, 1619, 433],
    "emprego": [24369, 24370, 24371, 24372, 24373, 24374,24375,24376,24377,24378,24379,24380,28543],
    "rendimento_habitual": [24382, 24383, 24384, 24385, 24386, 24387, 24388, 24399],
}

# Datas de início e fim para a consulta
data_inicial = "01/01/2015"
data_final = ""

# URL base da API
BASE_URL = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json&dataInicial={}&dataFinal={}"

In [107]:
# Definição das funções relevantes

# Função para pegar os dados da API SGS e trandormar em um dataframe
def obter_dados_api(chave):
    url = BASE_URL.format(chave, data_inicial, data_final)
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        return pd.DataFrame(data)
    else:
        print(f"Erro ao buscar dados para a chave {chave}")
        return None

# Função para fazer o processamento inicial dos dados (lidar com os valores nulls, modificar os tipos de variáveis e renomeação das colunas)
def processar_dados(df, chave):
    if df is None or df.empty:
        return None

    df["data"] = pd.to_datetime(df["data"], dayfirst=True)
    df["valor"] = pd.to_numeric(df["valor"], errors="coerce").fillna(0)
    df = df.rename(columns={"valor": str(chave)})

    return df

# Função que utiliza as anteriores para obter e processar um grupo de endpoints e criar um df concatenado de seu respectivo tema
def processar_grupo(endpoints):
    dfs = [processar_dados(obter_dados_api(chave), chave) for chave in endpoints]
    dfs = [df for df in dfs if df is not None]  # Remove valores None

    if dfs:
        # Garante que todos os DataFrames usem a coluna 'data' como índice
        for df in dfs:
            df.set_index("data", inplace=True)
        
        # Concatena mantendo um único índice (a data)
        return pd.concat(dfs, axis=1).reset_index()
    return None

# Função que executa o fluxo principal do código
def main():
    resultados = {}

    for tema, endpoints in grupos_endpoints.items():
        df_grupo = processar_grupo(endpoints)
        if df_grupo is not None:
            resultados[tema] = df_grupo
            print(f"✅ Dados do grupo '{tema}' processados com sucesso!")
        else:
            print(f"⚠️ Nenhum dado válido para o grupo '{tema}'.")

    return resultados



In [108]:
# Executar a função principal
if __name__ == "__main__":
    dataframes_por_tema = main()

    # Exemplo: Acessando um DataFrame específico
    print("\n📊 Exemplo de DataFrame para 'emprego':")
    print(dataframes_por_tema.get("emprego", "Nenhum dado encontrado."))

✅ Dados do grupo 'utilidades' processados com sucesso!
✅ Dados do grupo 'emprego' processados com sucesso!
✅ Dados do grupo 'rendimento_habitual' processados com sucesso!

📊 Exemplo de DataFrame para 'emprego':
          data  24369   24370  24371  24372  24373  24374  24375  24376  \
0   2015-01-01    6.9  159848  65074  11147  21248   3846  37542  10565   
1   2015-02-01    7.5  160018  64596  11061  21224   3871  37288  10391   
2   2015-03-01    8.0  160189  64137  11031  21244   3966  37051  10218   
3   2015-04-01    8.1  160354  64224  11159  21404   3932  37068  10191   
4   2015-05-01    8.3  160525  64165  11138  21521   3867  36979  10248   
..         ...    ...     ...    ...    ...    ...    ...    ...    ...   
117 2024-10-01    6.2  176568  72260  12840  25678   4340  39021  14433   
118 2024-11-01    6.1  176689  72387  12847  25861   4350  39139  14404   
119 2024-12-01    6.2  176810  72129  12751  26026   4346  39237  14209   
120 2025-01-01    6.5  176930  71505  1

In [139]:
# Realizando a conexão do Python com o PostgreSQL usando o sqlalchemy

USERNAME = "Case_trainee"
PASSWORD = "041700"
HOST = "localhost"  # Ou endereço do servidor
PORT = "5432"
DATABASE = "indicadores_teste"

# Criando a engine do SQLAlchemy
try:
    engine = create_engine(
        f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}",
        connect_args={"options": "-c client_encoding=UTF8"}
    )
    print("✅ Engine created successfully!")
except Exception as e:
    print(f"❌ Engine creation failed: {e}")



✅ Engine created successfully!


In [125]:
# Função para inserir DataFrame no PostgreSQL
def inserir_dataframe(df, tabela_nome):
    df.to_sql(tabela_nome, engine, if_exists='append', index=False)
    print(f"✅ Dados inseridos na tabela {tabela_nome}")